# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [10]:
# 10 countries with the lowest infant mortality rates
document = ET.parse( './data/mondial_database.xml' )
import numpy as np

data = document.getroot()
# print (data)
# for country in data.iter('country'):
#     print (country.attrib)

   


In [11]:
import pandas as pd
for country in data.iter('country'):
    if country.find('name') != None:
        if country.find('infant_mortality') != None:
            c.append(country.find('name').text)
            imr.append(float(country.find('infant_mortality').text))
        else:
            continue
    else:
        continue
table = {'country':c, 'infant mortality': imr}
top10 = pd.DataFrame(table).sort_values(by='infant mortality').head(10)
top10

,country,infant mortality
264,Monaco,1.81
36,Monaco,1.81
90,Japan,2.13
318,Japan,2.13
262,Norway,2.48
34,Norway,2.48
337,Bermuda,2.48
109,Bermuda,2.48
326,Singapore,2.53
98,Singapore,2.53


In [12]:
# 10 cities with the largest population
popArray=[]

for city in data.findall('country/city'):

    for population in city.findall('population'):
        popArray.append((int(population.text),city.find('name').text))

largest_population = pd.DataFrame(popArray,columns = ['population','city'] ).sort_values('population', ascending=False).head(25)    

top10 = largest_population['city'].drop_duplicates()
top10
# print(largest_population)
# print(top10)



430              Seoul
412         Al Qahirah
204            Bangkok
322          Hong Kong
229        Ho Chi Minh
554          Singapore
409    Al Iskandariyah
566         New Taipei
434              Busan
270          Pyongyang
Name: city, dtype: object

In [13]:
# 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

ethnic_group = []
total_population = []
ethnic_numbers = []


for country in data.findall('country'):
    for population in reversed(country.findall('population')):
        total_population.append(int(population.text))
        for ethnicity in country.findall('ethnicgroup'):
            ethnic_numbers.append((int(population.text), float(ethnicity.attrib['percentage']), ethnicity.text))
        break

d= pd.DataFrame(ethnic_numbers, columns=['population', 'percentage', 'ethnicity'])
d['Ethnic Population'] = (d.population * d.percentage)/100

d.groupby('ethnicity').sum().sort_values(by='Ethnic Population', ascending=False).head(10)

,population,percentage,Ethnic Population
ethnicity,,,
Han Chinese,1360720000,91.50,1.245059e+09
Indo-Aryan,1210854977,72.00,8.718156e+08
European,1157295639,970.82,4.948722e+08
African,975352746,1868.55,3.183251e+08
Dravidian,1210854977,25.00,3.027137e+08
Mestizo,279743964,870.70,1.577344e+08
Bengali,149772364,98.00,1.467769e+08
Russian,322438406,224.10,1.318570e+08
Japanese,127298000,99.40,1.265342e+08


In [21]:
# name and country of 
#a) longest river
name = []
length = []
source = []

document = ET.parse( './data/mondial_database.xml' )


for element in document.getiterator('river'):
    if element.find('length') !=None:
        name.append(element.find('name').text)
        length.append(np.float(element.find('length').text))
        
riverList ={'name':name,
        'length':length} 
river=pd.DataFrame(riverList,columns=['name','length']).sort_values(by='length',ascending=False).head(1)
river
    



,name,length
174,Amazonas,6448.0


In [15]:
# b) largest lake and 
lakeList=[]
for element in document.getiterator('lake'):
    for subelement in list(element):
        if subelement.tag == 'name':
            Lname = subelement.text
        if subelement.tag == 'area':
            Larea = float(subelement.text)
        if subelement.tag == 'located':
            Llocation = subelement.attrib['country']
    lakeList.append((Lname, Larea, Llocation))
lake = pd.DataFrame(lakeList, columns=['Name', 'Area', 'C_Code']).sort_values(by='Area', ascending=False).head(1)
lake

,Name,Area,C_Code
54,Caspian Sea,386400.0,TM
109,Lake Superior,82103.0,USA
81,Lake Victoria,68870.0,EAT
106,Lake Huron,59600.0,USA
108,Lake Michigan,57800.0,USA


In [23]:
document = ET.parse( './data/mondial_database.xml' )
doc = document.getroot()
airportList = []
for element in doc.findall('airport'):
    if (element.find('elevation').text != None and element.find('name') != None):
        airportList.append((float(element.find('elevation').text), element.find('name').text, element.get('country')))
aList = pd.DataFrame(airportList, columns=['Elevation', 'Name', 'Code']).sort_values(by='Elevation', ascending=False)
aList.head(1)

,Elevation,Name,Code
80,4063.0,El Alto Intl,BOL
